In [3]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\nawka\appdata\local\temp\pip-req-build-zdmf5woc
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369579 sha256=b6c10c4999738366868f0d02d426853b1d3d10f15a6b65708f01cec1d651ab5f
  Stored in directory: C:\Users\nawka\AppData\Local\Temp\pip-ephem-wheel-cache-ig3t4z7c\wheels\3f\7c\a4\9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\nawka\AppData\Local\Temp\pip-req-build-zdmf5woc'


In [4]:
import tensorflow as tf
import numpy as np
import clip


In [5]:
# Load CLIP model
clip_model, clip_preprocess = clip.load("ViT-B/32", device="cpu")

100%|███████████████████████████████████████| 338M/338M [01:07<00:00, 5.27MiB/s]


In [31]:
# Define Conditional Operator (CoOp) function
def coop(features1, features2):
    # Concatenate the features along the last dimension
    concatenated_features = tf.concat([features1, features2], axis=-1)
    return concatenated_features

# Define a simple Siamese network with CoOp
def siamese_network(image):
    input_shape = image.shape[1:]
    inputs = tf.keras.layers.Input(shape=input_shape)
    features = clip_model.encode_image(image)

    # Split the features
    features1, features2 = tf.split(features, 2, axis=0)

    # Apply CoOp
    combined_features = coop(features1, features2)

    # Fully connected layer for classification
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(combined_features)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [8]:
import requests
from PIL import Image
from io import BytesIO

In [12]:
import os
files = os.listdir("./")

In [13]:
from PIL import Image
import numpy as np

def get_image_as_np_array(file_path):
    img = Image.open(file_path)
    img_array = np.array(img)
    return img_array

# Example file paths for demonstration purposes
file_path1 = files[5]
file_path2 = files[1]

# Get numpy arrays for the images
image1_array = get_image_as_np_array(file_path1)
image2_array = get_image_as_np_array(file_path2)

# Now you have numpy arrays for both images
# You can perform any desired operations with these arrays


In [33]:
# Example usage
# Assuming you have images stored as numpy arrays in img1 and img2
# img1 = image1_array  # Load image 1
# img2 = image2_array  # Load image 2

img1 = Image.open(files[5])  # Load image 1
img2 = Image.open(files[1])  # Load image 2

# Preprocess images using CLIP preprocess function
img1_preprocessed = clip_preprocess(img1)
img2_preprocessed = clip_preprocess(img2)

# img1_preprocessed = get_image_as_np_array(img1_preprocessed)
# img2_preprocessed = get_image_as_np_array(img2_preprocessed)

# Expand dimensions to match CLIP input shape (batch_size, height, width, channels)
img1_preprocessed = tf.expand_dims(img1_preprocessed, axis=0)
img2_preprocessed = tf.expand_dims(img2_preprocessed, axis=0)

# Create Siamese network model
siamese_model = siamese_network(img1_preprocessed)

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with your dataset
# Assuming you have a dataset with pairs of images and their labels (0 for different persons, 1 for same person)
# x_train contains pairs of preprocessed images
# y_train contains corresponding labels
siamese_model.fit(x_train, y_train, epochs=10, batch_size=32)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'type'

In [30]:
img1_preprocessed

<tf.Tensor: shape=(1, 3, 224, 224), dtype=float32, numpy=
array([[[[1.9303361, 1.9303361, 1.9303361, ..., 1.9303361, 1.9303361,
          1.9303361],
         [1.9303361, 1.9303361, 1.9303361, ..., 1.9303361, 1.9303361,
          1.9303361],
         [1.9303361, 1.9303361, 1.9303361, ..., 1.9303361, 1.9303361,
          1.9303361],
         ...,
         [1.9303361, 1.9303361, 1.9303361, ..., 1.9303361, 1.9303361,
          1.9303361],
         [1.9303361, 1.9303361, 1.9303361, ..., 1.9303361, 1.9303361,
          1.9303361],
         [1.9303361, 1.9303361, 1.9303361, ..., 1.9303361, 1.9303361,
          1.9303361]],

        [[2.0748837, 2.0748837, 2.0748837, ..., 2.0748837, 2.0748837,
          2.0748837],
         [2.0748837, 2.0748837, 2.0748837, ..., 2.0748837, 2.0748837,
          2.0748837],
         [2.0748837, 2.0748837, 2.0748837, ..., 2.0748837, 2.0748837,
          2.0748837],
         ...,
         [2.0748837, 2.0748837, 2.0748837, ..., 2.0748837, 2.0748837,
          2.07